In [ ]:
# stored lists of alpha, beta, g, probability difference of ordinal patterns
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.integrate import solve_ivp
from scipy import integrate
from matplotlib.pyplot import figure
from itertools import repeat
import seaborn as sn
import ordpy
import pandas as pd
import array

alpha = 0
beta = 0
alpha_max = 0.04
beta_max = 0.04
beta_repeat_times = int((beta_max-beta)/0.0025)
g = 0
g_max = g+0.025*beta_repeat_times

alpha_array_layer_3 = []
alpha_array_layer_2 = []
alpha_array_layer_1 = []
beta_array_layer_2 = []
beta_array_layer_1 = []
g_array_layer_1 = []
probability_difference_list = []

while g < g_max:
    while beta < beta_max:
        while alpha < alpha_max:
            alpha_array_layer_3.append(alpha)
            alpha += 0.0025
        beta_array_layer_2.extend(repeat(beta, beta_repeat_times))
        alpha_array_layer_2.extend(alpha_array_layer_3)
        beta += 0.0025
    g_array_layer_1.extend(repeat(g, beta_repeat_times**2))
    alpha_array_layer_1.extend(alpha_array_layer_2)
    beta_array_layer_1.extend(beta_array_layer_2)
    g += 0.025
    
print(beta_repeat_times)
print(g_max)
print(len(alpha_array_layer_1))
print(len(beta_array_layer_1))
print(len(g_array_layer_1))
    
xi_0 = 0.175
T_1_ = 5
xi_P = 0.175
A_P = 0.01
omega_P = 0.7
A_0 = 0.5
omega_T_ = 0.7
c = 0.1
b_0 = 0.6
a = 0.7
T_2_ = 5

def JJ (t, y, xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g):
    x1, y1, x2, y2, z = y
    dx1 = x1*(1-xi_0*math.e**(-1/T_1_)-xi_P)-y1-1/3*x1**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_1_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy1 = c*(-b_0*math.e**(1/T_1_)*y1+a+x1)
    dx2 = x2*(1-xi_0*math.e**(-1/T_2_)-xi_P)-y2-1/3*x2**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_2_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy2 = c*(-b_0*math.e**(1/T_2_)*y2+a+x2)
    dz = g*(x1-x2)
    dydt = [dx1, dy1, dx2, dy2, dz]
    return dydt

tStart = 0
tEnd = 8000

X0 = [0.2, 0.1, 0.02, 0.01, 0.001]

k = 0
while k < len(alpha_array_layer_1):
    alpha = alpha_array_layer_1[k]
    beta = beta_array_layer_1[k]
    g = g_array_layer_1[k]
    
    solution = integrate.solve_ivp(JJ, [tStart, tEnd], X0, method='RK45', t_eval=np.linspace(tStart,tEnd,100000), args=(xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g))
    
    t_sol = solution.t
    x1_sol = solution.y[0]
    y1_sol = solution.y[1]
    x2_sol = solution.y[2]
    y2_sol = solution.y[3]
    z_sol = solution.y[4]

    # error function
    error_func = []
    i = 0
    while i<len(t_sol):
        error_func.append(math.sqrt((x1_sol[i]-x2_sol[i])**2+(y1_sol[i]-y2_sol[i])**2))
        i += 1
    
    # P(pattern0123) - P(pattern3210)
    ordinal_distribution = ordpy.ordinal_distribution(error_func, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)

    probability_difference=ordinal_distribution[1][0]-ordinal_distribution[1][len(ordinal_distribution[1])-1]

    probability_difference_list.append(probability_difference)
    
    k += 1

print(len(probability_difference_list))

import json

with open("alpha list", "w") as alpha_list:
    json.dump(alpha_array_layer_1, alpha_list)
with open("beta list", "w") as beta_list:
    json.dump(beta_array_layer_1, beta_list)
with open("g list", "w") as g_list:
    json.dump(g_array_layer_1, g_list)
with open("probability difference list", "w") as prob_dif_list:
    json.dump(probability_difference_list, prob_dif_list)

In [ ]:
# retrieving data
import json

with open("alpha list", "r") as alpha_list:
    alpha_list_opened = json.load(alpha_list)
with open("beta list", "r") as beta_list:
    beta_list_opened = json.load(beta_list)
with open("g list", "r") as g_list:
    g_list_opened = json.load(g_list)
with open("probability difference list", "r") as prob_dif_list:
    probability_difference_list_opened = json.load(prob_dif_list)

In [ ]:
# plotting 3D heatmap
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111, projection='3d')

img = ax.scatter(alpha_list_opened, beta_list_opened, g_list_opened, c=probability_difference_list_opened, marker=',', s=50)
ax.set_xlabel('alpha', fontsize=15)
ax.set_ylabel('beta', fontsize=15)
ax.set_zlabel('g', fontsize=15)
ax.set_title('difference between probability of pattern0123 and pattern3210')
ax.view_init(elev=0, azim=-90)
fig.colorbar(img)
plt.show()